In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/MyDrive/ML/Kaggle_Optiver/Data/"
%cd /content/drive/MyDrive/ML/Kaggle_Optiver/Data/

%run ./Functions.ipynb


Mounted at /content/drive
/content/drive/MyDrive/ML/Kaggle_Optiver/Data


In [10]:
train = pd.read_csv('./train.csv')
display(train.shape)
display(train.head(2))
display(train.time_id.nunique())

# stock_ids = set(train['stock_id'])
train_data = get_dataSet(stock_ids = train['stock_id'].unique(), dataType = 'train')
train_dataSet = pd.merge(train, train_data, on = ['stock_id', 'time_id'], how = 'left')

(428932, 3)

,stock_id,time_id,target
0,0,5,0.004136
1,0,11,0.001445


3830

In [11]:
display(train_dataSet.shape)
display(train_dataSet.head(2))
display(train_dataSet.time_id.nunique())
display(train_dataSet.stock_id.nunique())
# train_dataSet.to_csv("./train_dataSet.csv")

(428932, 5)

,stock_id,time_id,target,log_return,range_diff
0,0,5,0.004136,0.004115,0.000852
1,0,11,0.001445,0.001268,0.000394


3830

112

In [2]:
# train_dataSet = pd.read_csv('./train_dataSet.csv')
# display(train_dataSet.shape)
# display(train_dataSet.head(2))
# display(train_dataSet.time_id.nunique())
# display(train_dataSet.stock_id.nunique())

(428932, 6)

,Unnamed: 0,stock_id,time_id,target,log_return,range_diff
0,0,0,5,0.004136,0.004115,0.000852
1,1,0,11,0.001445,0.001268,0.000394


3830

112

In [3]:
y_train = train_dataSet['target']
X_train = train_dataSet.drop(['stock_id', 'time_id', 'target'], axis = 1)
X_train.head()

,Unnamed: 0,log_return,range_diff
0,0,0.004115,0.000852
1,1,0.001268,0.000394
2,2,0.002719,0.000725
3,3,0.002625,0.000861
4,4,0.001901,0.000397


In [4]:
params = {'lambda': 1.00952957200908,
         'alpha': 0.006342908262784082,
         'colsample_bytree': 1.0,
         'subsample': 0.4,
         'learning_rate': 0.02,
         'n_estimators': 617,
         'max_depth': 7,
         'random_state': 2020,
         'min_child_weight': 101}

clf = XGBRegressor(**params, tree_method='gpu_hist')
clf.fit(X_train,y_train.to_numpy().ravel())

XGBRegressor(alpha=0.006342908262784082, base_score=None, booster=None,
             callbacks=None, colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=1.0, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, lambda=1.00952957200908,
             learning_rate=0.02, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=7,
             max_leaves=None, min_child_weight=101, missing=nan,
             monotone_constraints=None, n_estimators=617, n_jobs=None,
             num_parallel_tree=None, ...)

In [5]:
test = pd.read_csv('./test.csv')
display(test.shape)
display(test)

(3, 3)

,stock_id,time_id,row_id
0,0,4,0-4
1,0,32,0-32
2,0,34,0-34


In [6]:
test_data = get_dataSet(stock_ids = test['stock_id'].unique(), dataType = 'test')
test_dataSet = pd.merge(test, test_data, on = ['stock_id', 'time_id'], how = 'left')
test_dataSet = test_dataSet.drop(['stock_id', 'time_id'], axis = 1)
display(test_dataSet.shape)
display(test_dataSet)

y_pred = test_dataSet[['row_id']]
X_test = test_dataSet.drop(['row_id'], axis = 1).fillna(0)

(3, 3)

,row_id,log_return,range_diff
0,0-4,0.000273,0.000557
1,0-32,NaN,NaN
2,0-34,NaN,NaN


In [ ]:
test_stock_stat_df = get_dataSet(stock_ids = test['stock_id'].unique(), dataType = 'test')
test_dataSet = pd.merge(test, test_stock_stat_df, on = ['stock_id', 'time_id'], how = 'left')
test_dataSet = test_dataSet.drop(['stock_id', 'time_id'], axis = 1)

y_pred = test_dataSet[['row_id']]
X_test = test_dataSet.drop(['row_id'], axis = 1).fillna(0)

y_pred = y_pred.assign(target = clf.predict(X_test))
# y_pred.to_csv('./submission.csv',index = False)

In [ ]:
y_pred

,row_id,target
0,0-4,0.001522
1,0-32,0.000662
2,0-34,0.000662
